In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql import functions as f
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import udf


spark = SparkSession.builder.appName("Exercise-1").getOrCreate()
df = spark.read.format("csv")\
  .option("header","true")\
  .option("inferSchema", "true")\
  .load("dbfs:/FileStore/shared_uploads/marcos.mussio@gmail.com/championships.csv")\
  .withColumnRenamed("Campeón","Campeon")
df.show()

+-------+-----------------+-----------------+
 Torneo| Campeon| Subcampeón|
+-------+-----------------+-----------------+
 2020| Nacional| Rentistas|
 2019| Nacional| Peñarol|
 2018| Peñarol| Nacional|
 2017| Peñarol|Defensor Sporting|
 2016| Nacional| Wanderers|
2015-16| Peñarol| Nacional|
2014-15| Nacional| Peñarol|
2013-14| Danubio| Wanderers|
2012-13| Peñarol|Defensor Sporting|
2011-12| Nacional| Peñarol|
2010-11| Nacional|Defensor Sporting|
2009-10| Peñarol| Nacional|
2008-09| Nacional|Defensor Sporting|
2007-08|Defensor Sporting| Peñarol|
2006-07| Danubio| Peñarol|
2005-06| Nacional| Rocha|
 2005| Nacional|Defensor Sporting|
 2004| Danubio| Nacional|
 2003| Peñarol| Nacional|
 2002| Nacional| Danubio|
+-------+-----------------+-----------------+
only showing top 20 rows

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

windowSpec  = Window.partitionBy(df.Campeon).orderBy(df.Torneo)

dfn = df.withColumn("row_number",row_number()\
  .over(windowSpec))\
  .orderBy("row_number", ascending=False)
    
w = Window.partitionBy('Campeon')
dfn.withColumn('maxStreak', f.max('row_number')\
  .over(w))\
  .where(f.col('row_number') == f.col('maxStreak'))\
  .drop('row_number')\
  .drop('Subcampeón')\
  .show()


+-------+--------------------+---------+
 Torneo| Campeon|maxStreak|
+-------+--------------------+---------+
 1990| Bella Vista| 1|
 1911| CURCC| 5|
 1984| Central Español| 1|
2013-14| Danubio| 4|
2007-08| Defensor Sporting| 4|
 2020| Nacional| 48|
 1948| No Completado| 2|
 1930| No Jugado| 2|
 1926|No Jugado (un tor...| 1|
 2018| Peñarol| 46|
 1989| Progreso| 1|
 1927| Rampla Juniors| 1|
 1914| River Plate FC| 4|
 1931| Wanderers| 4|
+-------+--------------------+---------+